In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import trange # to measure time

Before crawling, check **/robots.txt** from domain url.

(ex. movie.naver.com//robots.txt)

In chrome dev tool, click 'copy selector' over the element which you want to crawl

main > section > div > div > div.chitchats > div.chitchat-list > div:nth-child(2) > div > div.body.edit-chitchat

In [68]:
base_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=151728&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false"
base_url = base_url + "&page="

comments = []

def get_by_page(pagenum):
    url = base_url + str(pagenum)
    response = requests.get(url) # response is 200 if normal
    soup = bs(response.content, 'html.parser') # loads every element in html page
    content = soup.select(".score_result .score_reple")
    
    for i in range(10): # 10 comments in a page
        comments.append(content[i].select("p")[0].get_text(strip=True))
        if pagenum == 1: # for test
            print(comments[-1])
            print('=========================')


In [69]:
for i in trange(1,101):
    get_by_page(i)

print(len(comments))

  2%|▏         | 2/100 [00:00<00:07, 13.16it/s]

먼저 떠난 우리 아들도 저렇게 멋진데서 잘 지내길..평생 기억할께 꼭 다시 만나자
'신과 함께'와는 격이 다른 저승영화
박평식 : "따뜻해, 삶을 껴안고 다독이는 죽음" 박평식 평론가의 평점이 이렇게 공감된다니... 2018 최고의 애니메이션? 아니 이건 최고의 영화
관람객사람이 진정으로 죽는 순간은 사랑하는 사람에게 잊혀졌을때..사랑하는 사람들을 영원히 기억하고 나 또한 그 사람들에게 영원히 기억되길..Remember me..
사람은 3번 죽는다.숨이 멎는 순간 생물학 적으로 죽고, 장례식에 온 하객들이 떠나갈  때 사회적으로 죽고 그 사람을 기억하는 마지막 사람이 죽으면 그것이 진정한 죽음이라는것을 코코를 보고 알게되었다.
내일 죽는다면 오늘 봐야할 영화
마지막에 코코 할머니 손잡고 울면서 기억해줘 노래부를때 정말 심각하게 슬펐네요...ㅠㅠ
픽사는 우리를 배신하지 않아...
remember me... 내가 나중에 죽는다면 사후세계에 갔을때 울엄마아빠가 나 마중나와 있어주면 좋겠다.... 엄마아빠 사랑해
픽사 역대급 애니다... 평식이도 8점줬다? 긴말 필요없다 이건 영화관 꼭 가서 보즈아!!!


100%|██████████| 100/100 [00:07<00:00, 12.87it/s]

1000


In [70]:
orgdf = pd.DataFrame({"text": comments})
orgdf.shape

(1000, 1)

In [72]:
orgdf.to_csv("data/coco-movie-review.csv", index=False)

In [73]:
df = pd.read_csv("data/coco-movie-review.csv")

In [74]:
import numpy as np
import re
import matplotlib.pyplot as plt
#matplotlib settings
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.rc("font", family="Malgun Gothic")
plt.rc('axes', unicode_minus=False)

In [75]:
# delete duplicated comments. keeps last one
print(df.shape)
df = df.drop_duplicates(["text"], keep="last")
print(df.shape)

(1000, 1)
(995, 1)


In [76]:
df["origin_text"] = df["text"]
df.head()

,text,origin_text
0,먼저 떠난 우리 아들도 저렇게 멋진데서 잘 지내길..평생 기억할께 꼭 다시 만나자,먼저 떠난 우리 아들도 저렇게 멋진데서 잘 지내길..평생 기억할께 꼭 다시 만나자
1,'신과 함께'와는 격이 다른 저승영화,'신과 함께'와는 격이 다른 저승영화
2,"박평식 : ""따뜻해, 삶을 껴안고 다독이는 죽음"" 박평식 평론가의 평점이 이렇게 공...","박평식 : ""따뜻해, 삶을 껴안고 다독이는 죽음"" 박평식 평론가의 평점이 이렇게 공..."
3,관람객사람이 진정으로 죽는 순간은 사랑하는 사람에게 잊혀졌을때..사랑하는 사람들을 ...,관람객사람이 진정으로 죽는 순간은 사랑하는 사람에게 잊혀졌을때..사랑하는 사람들을 ...
4,"사람은 3번 죽는다.숨이 멎는 순간 생물학 적으로 죽고, 장례식에 온 하객들이 떠나...","사람은 3번 죽는다.숨이 멎는 순간 생물학 적으로 죽고, 장례식에 온 하객들이 떠나..."


In [98]:
df.dropna().shape

(994, 12)

In [99]:
df["text"] = df["text"].str.lower()

In [100]:
search_keyword = ['감동', '사랑', '죽음', '울', 
                  '할머니', '아이', '노래', '픽사', 
                  '추억', '눈물']
for keyword in search_keyword:
    df[keyword] = df["text"].str.contains(keyword)
df.head()

,text,origin_text,감동,사랑,죽음,울,할머니,아이,노래,픽사,추억,눈물
0,먼저 떠난 우리 아들도 저렇게 멋진데서 잘 지내길..평생 기억할께 꼭 다시 만나자,먼저 떠난 우리 아들도 저렇게 멋진데서 잘 지내길..평생 기억할께 꼭 다시 만나자,False,False,False,False,False,False,False,False,False,False
1,'신과 함께'와는 격이 다른 저승영화,'신과 함께'와는 격이 다른 저승영화,False,False,False,False,False,False,False,False,False,False
2,"박평식 : ""따뜻해, 삶을 껴안고 다독이는 죽음"" 박평식 평론가의 평점이 이렇게 공...","박평식 : ""따뜻해, 삶을 껴안고 다독이는 죽음"" 박평식 평론가의 평점이 이렇게 공...",False,False,True,False,False,False,False,False,False,False
3,관람객사람이 진정으로 죽는 순간은 사랑하는 사람에게 잊혀졌을때..사랑하는 사람들을 ...,관람객사람이 진정으로 죽는 순간은 사랑하는 사람에게 잊혀졌을때..사랑하는 사람들을 ...,False,True,False,False,False,False,False,False,False,False
4,"사람은 3번 죽는다.숨이 멎는 순간 생물학 적으로 죽고, 장례식에 온 하객들이 떠나...","사람은 3번 죽는다.숨이 멎는 순간 생물학 적으로 죽고, 장례식에 온 하객들이 떠나...",False,False,True,False,False,False,False,False,False,False


In [101]:
df["text"].str.contains("죽음")

0      False
1      False
2       True
3      False
4       True
       ...  
995    False
996    False
997    False
998    False
999     True
Name: text, Length: 995, dtype: object

In [103]:
# setting flag na=False to prevent error:
# Cannot mask with non-boolean array containing NA
df_python = df[df["text"].str.contains("죽음|추억", na = False)].copy()
df_python.shape

(57, 12)

In [104]:
df[search_keyword]

,감동,사랑,죽음,울,할머니,아이,노래,픽사,추억,눈물
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
995,False,False,False,True,False,False,False,True,False,False
996,True,False,False,True,False,False,False,False,False,False
997,False,False,False,False,False,True,False,False,False,False
998,True,False,False,False,False,False,False,False,False,True


In [105]:
df[search_keyword].sum().sort_values(ascending=False)

감동     216.0
울      196.0
눈물     128.0
아이      66.0
노래      62.0
픽사      60.0
죽음      55.0
사랑      53.0
할머니     51.0
추억       4.0
dtype: float64

In [106]:
text = df.loc[(df["추억"] == True), "text"]
for t in text:
    print("-"*20)
    print(t)

--------------------
이 세상에서 더 이상 나를 기억하고 추억하는 이가 없다면 사후세계에서조차 사라지는 영원한 죽음을 맞이한다는게 서글펐다. 멕시코와 한국의 조상에 대한 정서와 추모 문화가 비슷하다는게 인상적.
--------------------
관람객리멤버미...너무나도 큰 감동을 얻고 울면서 극장을 나왔네요 문화는 달라도 죽은 이를 추억하는 그 마음은 동일함에 크나큰 감동을 느꼈습니다 여러번 볼 생각이에요 정말 좋은 영화입니다 :)
--------------------
떠난 사람들도 떠나 보낸 사람들도 기억해줘요... 이 순간에도 추억이라는 따뜻한 얼굴로 가슴속에 남아 있다는 것을.
--------------------
죽음을 넘어선 그 곳에서 다시 한 번 가족과 용서와 사랑을. 추억을.
